In [1]:
%matplotlib inline
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim    

from Fairness_Metrics import fairness_metric
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('csv/IA_counties.csv')
data.head()

,X,Y,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,...,PRES08OTH,TOTVOT12,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD
0,-92.999001,42.035801,19,127,19127,Marshall,Marshall County,1482770678,1803086,42.041691,...,356,19064,10257,8472,335,17980,7652,9146,1182,1
1,-92.066966,42.080299,19,11,19011,Benton,Benton County,1855117342,5760770,42.092547,...,207,14023,6862,6940,221,13844,4678,8232,934,1
2,-95.150846,43.082520,19,41,19041,Clay,Clay County,1469139214,13866941,43.079822,...,118,8502,3385,4951,166,8617,2249,5877,491,4
3,-95.325464,41.684636,19,165,19165,Shelby,Shelby County,1530110414,1486135,41.679014,...,102,6483,2469,3911,103,6370,1662,4362,346,4
4,-91.352586,42.862146,19,43,19043,Clayton,Clayton County,2016405612,36586071,42.840998,...,143,9138,4806,4164,168,9129,3237,5317,575,1


In [4]:
data.columns

Index(['X', 'Y', 'STATEFP10', 'COUNTYFP10', 'GEOID10', 'NAME10', 'NAMELSAD10',
       'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'TOTPOP', 'NH_WHITE',
       'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE',
       'HISP', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'N_NHPI', 'H_OTHER',
       'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP',
       'NHPIVAP', 'OTHERVAP', '2MOREVAP', 'TOTVOT00', 'PRES00D', 'PRES00R',
       'PRES00G', 'PRES00OTH', 'TOTVOT04', 'PRES04D', 'PRES04R', 'PRES04OTH',
       'TOTVOT08', 'PRES08D', 'PRES08R', 'PRES08OTH', 'TOTVOT12', 'PRES12D',
       'PRES12R', 'PRES12OTH', 'TOTVOT16', 'PRES16D', 'PRES16R', 'PRES16OTH',
       'CD'],
      dtype='object')

In [5]:
data[['TOTVOT16', 'PRES16D', 'PRES16R']]

,TOTVOT16,PRES16D,PRES16R
0,17980,7652,9146
1,13844,4678,8232
2,8617,2249,5877
3,6370,1662,4362
4,9129,3237,5317
...,...,...,...
94,16151,6215,8803
95,50667,25709,19458
96,6872,1856,4527
97,42680,15355,24447


In [6]:
req_data = pd.DataFrame({})
req_data['d_votes'] = data['PRES16D'] / data['TOTVOT16']
req_data['r_votes'] = data['PRES16R'] / data['TOTVOT16']
req_data['CD'] = data['CD']
req_data.head()

,d_votes,r_votes,CD
0,0.425584,0.508676,1
1,0.337908,0.594626,1
2,0.260996,0.682024,4
3,0.260911,0.684772,4
4,0.354584,0.582430,1


In [7]:
req_data['d_wasted'] = req_data['d_votes']
req_data['r_wasted'] = req_data['r_votes']
for index in range(len(req_data)):
    if req_data['d_votes'][index] > 0.5:
        req_data['d_wasted'][index] = req_data['d_votes'][index] - 0.5
        req_data['r_wasted'][index] = req_data['r_votes'][index]
        
    elif req_data['r_votes'][index] > 0.5:
        req_data['r_wasted'][index] = req_data['r_votes'][index] - 0.5
        req_data['d_wasted'][index] = req_data['d_votes'][index]
        
req_data.head()

,d_votes,r_votes,CD,d_wasted,r_wasted
0,0.425584,0.508676,1,0.425584,0.008676
1,0.337908,0.594626,1,0.337908,0.094626
2,0.260996,0.682024,4,0.260996,0.182024
3,0.260911,0.684772,4,0.260911,0.184772
4,0.354584,0.582430,1,0.354584,0.082430


In [8]:
fairness_metric(req_data)

0.3149162131754735

In [13]:
def customLoss(my_outputs):    # Work needed
    return fairness_metric(my_outputs)

## Reference: Deeplizard course (https://deeplizard.com/learn/video/jkdXDinWfo8)

Could not get the Reinforcement Learning working since I could not build the custom environment

In [14]:
# Neural Network for Reinforcement Learning

class DQN(nn.Module):
    def __init__(self, state):
        super().__init__()

        self.fc1 = nn.Linear(in_features = 4 , out_features = 24)   
        self.fc2 = nn.Linear(in_features = 24, out_features = 32)
        self.out = nn.Linear(in_features = 32, out_features = 4)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = customLoss(state)
        self.device = 'cuda'
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.out(x)

        return actions

In [18]:
# Agent to select actions for the RL model

class Agent():
    def __init__(self, num_actions):
        self.current_step = 0
        self.num_actions = num_actions
        
    def select_action(self, state, policy_net):
        rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * self.current_step)
        self.current_step += 1

        if rate > random.random():
            action = random.randrange(self.num_actions)
            return torch.tensor([action])                    # exploration
        else:
            with torch.no_grad():
                return policy_net(state)                     # exploitation

In [19]:
batch_size = 10      # To consider 10 states at a time
gamma = 0.999        # Discounted factor
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001
eps_decay = 0.001    # Delay value of epsilon after each episode
target_update = 10   # Update the weights of the target network after every 10 episodes
memory_size = 100000 # Memory size of the model
lr = 0.001           # Learning rate
num_episodes = 500   # Total number of episodes

In [20]:
device = "cuda"
class RedistrictingEnvManager():           # Incomplete environment 
    def __init__(self, req_data):
        self.action_space = [1, 2, 3, 4]   # For 4 Congressional Districts
        self.states = req_data             # For 99 VTDs
    
    def reset(self):
        self.state = 0
        
    def step(self, state, action):
        return state+1, fairness_metric(self.states[state])

In [21]:
em = RedistrictingEnvManager()

In [22]:
agent = Agent(4)
memory = Memory(memory_size)

In [23]:
policy_net = DQN().to(device)

In [25]:
optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)

In [26]:
def extract_tensors(experiences):
    # Convert batch of Experiences to Experience of batches
    batch = Experience(*zip(*experiences))

    t1 = torch.cat(batch.state)
    t2 = torch.cat(batch.action)
    t3 = torch.cat(batch.reward)
    t4 = torch.cat(batch.next_state)

    return (t1,t2,t3,t4)

In [28]:
class QValues():
    # Returns predicted q-values of the network on the given state-action pair
    def get_current(policy_net, states, actions):
        return policy_net(states).gather(dim=1, index=actions)            
    
    # Returns the maximum q-values of all the states
    def get_next(policy_net, next_states):                
        final_state_locations = 99
        non_final_state_locations = [i for i in range(1,99)]
        non_final_states = next_states[non_final_state_locations]
        batch_size = next_states.shape[0]
        values = torch.zeros(batch_size)
        values[non_final_state_locations] = policy_net(non_final_states).max(dim = 1)
        return values

In [30]:
def plot(values, moving_avg_period):
    plt.figure(2)
    plt.clf()        
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(values)

    moving_avg = get_moving_average(moving_avg_period, values)
    plt.plot(moving_avg)    
    plt.pause(0.001)
    print("Episode", len(values), "\n", \
        moving_avg_period, "episode moving avg:", moving_avg[-1])
    if is_ipython: display.clear_output(wait=True)

In [31]:
episode_durations = []

In [ ]:
for episode in range(num_episodes):
    em.reset()
    state = em.get_state()
    
    for timestep in count():
        action = agent.select_action(state, policy_net)
#         reward = em.take_action(action)
#         next_state = em.get_state()
#         memory.push(Experience(state, action, next_state, reward))
#         state = next_state
    
#         if memory.can_provide_sample(batch_size):
#             experiences = memory.sample(batch_size)
#             states, actions, rewards, next_states = extract_tensors(experiences)

#             current_q_values = QValues.get_current(policy_net, states, actions)
#             next_q_values = QValues.get_next(target_net, next_states)
#             target_q_values = (next_q_values * gamma) + rewards

#             loss = customLoss(current_q_values, target_q_values)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#         if em.done:
#             episode_durations.append(timestep)
#             plot(episode_durations, 100)
#             break

#     if episode % target_update == 0:
#             target_net.load_state_dict(policy_net.state_dict())
        
em.close()